In [34]:
# data
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected, batch_norm, dropout
from tensorflow.contrib.framework import arg_scope
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#import xgboost as xgb

# tqdm
from tqdm import tqdm_notebook as tqdm

tf.set_random_seed(777)  # reproducibility

# layers
from tensorflow.contrib.layers import fully_connected, batch_norm, dropout
from tensorflow.contrib.framework import arg_scope

# miscellaneous
from datetime import datetime
from time import time

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
result_df = pd.read_csv('./data/sampleSubmission.csv')

In [3]:
train_df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [4]:
test_df.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [5]:
result_df.head()

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
train_df.describe()

,X,Y
count,878049.000000,878049.000000
mean,-122.422616,37.771020
std,0.030354,0.456893
min,-122.513642,37.707879
25%,-122.432952,37.752427
50%,-122.416420,37.775421
75%,-122.406959,37.784369
max,-120.500000,90.000000


In [7]:
train_df.describe(include=['O'])

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address
count,878049,878049,878049,878049,878049,878049,878049
unique,389257,39,879,7,10,17,23228
top,2011-01-01 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,SOUTHERN,NONE,800 Block of BRYANT ST
freq,185,174900,60022,133734,157182,526790,26533


In [8]:
train_labels = train_df[['Category', 'Descript', 'Resolution']]
train_df_concat = train_df.drop(['Category', 'Descript', 'Resolution'], axis=1)

train_labels

,Category,Descript,Resolution
0,WARRANTS,WARRANT ARREST,"ARREST, BOOKED"
1,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,"ARREST, BOOKED"
2,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,"ARREST, BOOKED"
3,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,NONE
4,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,NONE
5,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,NONE
6,VEHICLE THEFT,STOLEN AUTOMOBILE,NONE
7,VEHICLE THEFT,STOLEN AUTOMOBILE,NONE
8,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,NONE
9,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,NONE


In [9]:
set(train_df['Category'])
sorted(train_df['Category'].unique())

['ARSON',
 'ASSAULT',
 'BAD CHECKS',
 'BRIBERY',
 'BURGLARY',
 'DISORDERLY CONDUCT',
 'DRIVING UNDER THE INFLUENCE',
 'DRUG/NARCOTIC',
 'DRUNKENNESS',
 'EMBEZZLEMENT',
 'EXTORTION',
 'FAMILY OFFENSES',
 'FORGERY/COUNTERFEITING',
 'FRAUD',
 'GAMBLING',
 'KIDNAPPING',
 'LARCENY/THEFT',
 'LIQUOR LAWS',
 'LOITERING',
 'MISSING PERSON',
 'NON-CRIMINAL',
 'OTHER OFFENSES',
 'PORNOGRAPHY/OBSCENE MAT',
 'PROSTITUTION',
 'RECOVERED VEHICLE',
 'ROBBERY',
 'RUNAWAY',
 'SECONDARY CODES',
 'SEX OFFENSES FORCIBLE',
 'SEX OFFENSES NON FORCIBLE',
 'STOLEN PROPERTY',
 'SUICIDE',
 'SUSPICIOUS OCC',
 'TREA',
 'TRESPASS',
 'VANDALISM',
 'VEHICLE THEFT',
 'WARRANTS',
 'WEAPON LAWS']

In [10]:
test_id = test_df.pop('Id')
test_df

,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
5,2015-05-10 23:40:00,Sunday,TARAVAL,BROAD ST / CAPITOL AV,-122.459024,37.713172
6,2015-05-10 23:30:00,Sunday,INGLESIDE,100 Block of CHENERY ST,-122.425616,37.739351
7,2015-05-10 23:30:00,Sunday,INGLESIDE,200 Block of BANKS ST,-122.412652,37.739750
8,2015-05-10 23:10:00,Sunday,MISSION,2900 Block of 16TH ST,-122.418700,37.765165
9,2015-05-10 23:10:00,Sunday,CENTRAL,TAYLOR ST / GREEN ST,-122.413935,37.798886


In [11]:
features = pd.concat([train_df_concat, test_df], keys = ['train_df', 'test_df'])
features

Dates  DayOfWeek  PdDistrict  \
train_df 0       2015-05-13 23:53:00  Wednesday    NORTHERN   
         1       2015-05-13 23:53:00  Wednesday    NORTHERN   
         2       2015-05-13 23:33:00  Wednesday    NORTHERN   
         3       2015-05-13 23:30:00  Wednesday    NORTHERN   
         4       2015-05-13 23:30:00  Wednesday        PARK   
         5       2015-05-13 23:30:00  Wednesday   INGLESIDE   
         6       2015-05-13 23:30:00  Wednesday   INGLESIDE   
         7       2015-05-13 23:30:00  Wednesday     BAYVIEW   
         8       2015-05-13 23:00:00  Wednesday    RICHMOND   
         9       2015-05-13 23:00:00  Wednesday     CENTRAL   
         10      2015-05-13 22:58:00  Wednesday     CENTRAL   
         11      2015-05-13 22:30:00  Wednesday     TARAVAL   
         12      2015-05-13 22:30:00  Wednesday  TENDERLOIN   
         13      2015-05-13 22:06:00  Wednesday    NORTHERN   
         14      2015-05-13 22:00:00  Wednesday     BAYVIEW   
         15      2015-05-13 22:00:00  Wednesday     BAYVIEW   
         16      2015-05-13 22:00:00  Wednesday  TENDERLOIN   
         17      2015-05-13 21:55:00  Wednesday   INGLESIDE   
         18      2015-05-13 21:40:00  Wednesday     BAYVIEW   
         19      2015-05-13 21:30:00  Wednesday  TENDERLOIN   
         20      2015-05-13 21:30:00  Wednesday   INGLESIDE   
         21      2015-05-13 21:17:00  Wednesday   INGLESIDE   
         22      2015-05-13 21:11:00  Wednesday  TENDERLOIN   
         23      2015-05-13 21:11:00  Wednesday  TENDERLOIN   
         24      2015-05-13 21:10:00  Wednesday    NORTHERN   
         25      2015-05-13 21:00:00  Wednesday  TENDERLOIN   
         26      2015-05-13 21:00:00  Wednesday    NORTHERN   
         27      2015-05-13 21:00:00  Wednesday   INGLESIDE   
         28      2015-05-13 21:00:00  Wednesday     TARAVAL   
         29      2015-05-13 20:56:00  Wednesday     TARAVAL   
...                              ...        ...         ...   
test_df  884232  2003-01-01 00:01:00  Wednesday     MISSION   
         884233  2003-01-01 00:01:00  Wednesday    SOUTHERN   
         884234  2003-01-01 00:01:00  Wednesday    RICHMOND   
         884235  2003-01-01 00:01:00  Wednesday   INGLESIDE   
         884236  2003-01-01 00:01:00  Wednesday     MISSION   
         884237  2003-01-01 00:01:00  Wednesday     CENTRAL   
         884238  2003-01-01 00:01:00  Wednesday     TARAVAL   
         884239  2003-01-01 00:01:00  Wednesday     MISSION   
         884240  2003-01-01 00:01:00  Wednesday    RICHMOND   
         884241  2003-01-01 00:01:00  Wednesday     BAYVIEW   
         884242  2003-01-01 00:01:00  Wednesday     MISSION   
         884243  2003-01-01 00:01:00  Wednesday     MISSION   
         884244  2003-01-01 00:01:00  Wednesday     BAYVIEW   
         884245  2003-01-01 00:01:00  Wednesday  TENDERLOIN   
         884246  2003-01-01 00:01:00  Wednesday     BAYVIEW   
         884247  2003-01-01 00:01:00  Wednesday     MISSION   
         884248  2003-01-01 00:01:00  Wednesday     CENTRAL   
         884249  2003-01-01 00:01:00  Wednesday     CENTRAL   
         884250  2003-01-01 00:01:00  Wednesday    SOUTHERN   
         884251  2003-01-01 00:01:00  Wednesday     MISSION   
         884252  2003-01-01 00:01:00  Wednesday   INGLESIDE   
         884253  2003-01-01 00:01:00  Wednesday     CENTRAL   
         884254  2003-01-01 00:01:00  Wednesday     CENTRAL   
         884255  2003-01-01 00:01:00  Wednesday   INGLESIDE   
         884256  2003-01-01 00:01:00  Wednesday     MISSION   
         884257  2003-01-01 00:01:00  Wednesday     MISSION   
         884258  2003-01-01 00:01:00  Wednesday    NORTHERN   
         884259  2003-01-01 00:01:00  Wednesday   INGLESIDE   
         884260  2003-01-01 00:01:00  Wednesday     BAYVIEW   
         884261  2003-01-01 00:01:00  Wednesday     TARAVAL   

                                       Address           X          Y  
train_df 0                  OAK ST / LAGUNA ST -122.

In [12]:
dateTime = pd.to_datetime(features['Dates'], format='%Y-%m-%d %H:%M:%S')
YMD = pd.concat([dateTime.dt.year, dateTime.dt.month, dateTime.dt.day], axis=1, keys=['Year', 'Month', 'Day'])
HMS = pd.concat([dateTime.dt.hour, dateTime.dt.minute, dateTime.dt.second], axis=1, keys=['hour', 'minute', 'second'])

In [13]:
features = pd.concat([features, YMD, HMS], axis=1)
features

Dates  DayOfWeek  PdDistrict  \
train_df 0       2015-05-13 23:53:00  Wednesday    NORTHERN   
         1       2015-05-13 23:53:00  Wednesday    NORTHERN   
         2       2015-05-13 23:33:00  Wednesday    NORTHERN   
         3       2015-05-13 23:30:00  Wednesday    NORTHERN   
         4       2015-05-13 23:30:00  Wednesday        PARK   
         5       2015-05-13 23:30:00  Wednesday   INGLESIDE   
         6       2015-05-13 23:30:00  Wednesday   INGLESIDE   
         7       2015-05-13 23:30:00  Wednesday     BAYVIEW   
         8       2015-05-13 23:00:00  Wednesday    RICHMOND   
         9       2015-05-13 23:00:00  Wednesday     CENTRAL   
         10      2015-05-13 22:58:00  Wednesday     CENTRAL   
         11      2015-05-13 22:30:00  Wednesday     TARAVAL   
         12      2015-05-13 22:30:00  Wednesday  TENDERLOIN   
         13      2015-05-13 22:06:00  Wednesday    NORTHERN   
         14      2015-05-13 22:00:00  Wednesday     BAYVIEW   
         15      2015-05-13 22:00:00  Wednesday     BAYVIEW   
         16      2015-05-13 22:00:00  Wednesday  TENDERLOIN   
         17      2015-05-13 21:55:00  Wednesday   INGLESIDE   
         18      2015-05-13 21:40:00  Wednesday     BAYVIEW   
         19      2015-05-13 21:30:00  Wednesday  TENDERLOIN   
         20      2015-05-13 21:30:00  Wednesday   INGLESIDE   
         21      2015-05-13 21:17:00  Wednesday   INGLESIDE   
         22      2015-05-13 21:11:00  Wednesday  TENDERLOIN   
         23      2015-05-13 21:11:00  Wednesday  TENDERLOIN   
         24      2015-05-13 21:10:00  Wednesday    NORTHERN   
         25      2015-05-13 21:00:00  Wednesday  TENDERLOIN   
         26      2015-05-13 21:00:00  Wednesday    NORTHERN   
         27      2015-05-13 21:00:00  Wednesday   INGLESIDE   
         28      2015-05-13 21:00:00  Wednesday     TARAVAL   
         29      2015-05-13 20:56:00  Wednesday     TARAVAL   
...                              ...        ...         ...   
test_df  884232  2003-01-01 00:01:00  Wednesday     MISSION   
         884233  2003-01-01 00:01:00  Wednesday    SOUTHERN   
         884234  2003-01-01 00:01:00  Wednesday    RICHMOND   
         884235  2003-01-01 00:01:00  Wednesday   INGLESIDE   
         884236  2003-01-01 00:01:00  Wednesday     MISSION   
         884237  2003-01-01 00:01:00  Wednesday     CENTRAL   
         884238  2003-01-01 00:01:00  Wednesday     TARAVAL   
         884239  2003-01-01 00:01:00  Wednesday     MISSION   
         884240  2003-01-01 00:01:00  Wednesday    RICHMOND   
         884241  2003-01-01 00:01:00  Wednesday     BAYVIEW   
         884242  2003-01-01 00:01:00  Wednesday     MISSION   
         884243  2003-01-01 00:01:00  Wednesday     MISSION   
         884244  2003-01-01 00:01:00  Wednesday     BAYVIEW   
         884245  2003-01-01 00:01:00  Wednesday  TENDERLOIN   
         884246  2003-01-01 00:01:00  Wednesday     BAYVIEW   
         884247  2003-01-01 00:01:00  Wednesday     MISSION   
         884248  2003-01-01 00:01:00  Wednesday     CENTRAL   
         884249  2003-01-01 00:01:00  Wednesday     CENTRAL   
         884250  2003-01-01 00:01:00  Wednesday    SOUTHERN   
         884251  2003-01-01 00:01:00  Wednesday     MISSION   
         884252  2003-01-01 00:01:00  Wednesday   INGLESIDE   
         884253  2003-01-01 00:01:00  Wednesday     CENTRAL   
         884254  2003-01-01 00:01:00  Wednesday     CENTRAL   
         884255  2003-01-01 00:01:00  Wednesday   INGLESIDE   
         884256  2003-01-01 00:01:00  Wednesday     MISSION   
         884257  2003-01-01 00:01:00  Wednesday     MISSION   
         884258  2003-01-01 00:01:00  Wednesday    NORTHERN   
         884259  2003-01-01 00:01:00  Wednesday   INGLESIDE   
         884260  2003-01-01 00:01:00  Wednesday     BAYVIEW   
         884261  2003-01-01 00:01:00  Wednesday     TARAVAL   

                                       Address           X          Y  Year  \
train_df 0                  OAK ST / LAGUNA S

In [14]:
features = features.drop(['Dates'], axis=1)

In [15]:
features['Address'].value_counts()

800 Block of BRYANT ST                   53517
800 Block of MARKET ST                   13464
2000 Block of MISSION ST                 10052
1000 Block of POTRERO AV                  8210
0 Block of TURK ST                        6444
900 Block of MARKET ST                    6256
0 Block of 6TH ST                         5682
300 Block of ELLIS ST                     5488
16TH ST / MISSION ST                      5156
400 Block of ELLIS ST                     5026
1000 Block of MARKET ST                   4924
1100 Block of MARKET ST                   4563
100 Block of OFARRELL ST                  4464
2000 Block of MARKET ST                   4433
700 Block of MARKET ST                    4062
3200 Block of 20TH AV                     4036
100 Block of 6TH ST                       3858
500 Block of JOHNFKENNEDY DR              3728
0 Block of UNITEDNATIONS PZ               3650
0 Block of PHELAN AV                      3509
200 Block of TURK ST                      3477
TURK ST / TAY

In [16]:
features.describe(include=['O'])

,DayOfWeek,PdDistrict,Address
count,1762311,1762311,1762311
unique,7,10,24777
top,Friday,SOUTHERN,800 Block of BRYANT ST
freq,268437,314638,53517


In [17]:
features.describe()

,X,Y,Year,Month,Day,hour,minute,second
count,1.762311e+06,1.762311e+06,1.762311e+06,1.762311e+06,1.762311e+06,1.762311e+06,1.762311e+06,1762311.0
mean,-1.224227e+02,3.777125e+01,2.008708e+03,6.375071e+00,1.561060e+01,1.340431e+01,2.015405e+01,0.0
std,3.067205e-02,4.711145e-01,3.634536e+00,3.448681e+00,8.829256e+00,6.554258e+00,1.860271e+01,0.0
min,-1.225136e+02,3.770788e+01,2.003000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,-1.224330e+02,3.775239e+01,2.006000e+03,3.000000e+00,8.000000e+00,9.000000e+00,0.000000e+00,0.0
50%,-1.224165e+02,3.777542e+01,2.009000e+03,6.000000e+00,1.600000e+01,1.400000e+01,1.900000e+01,0.0
75%,-1.224070e+02,3.778436e+01,2.012000e+03,9.000000e+00,2.300000e+01,1.900000e+01,3.300000e+01,0.0
max,-1.205000e+02,9.000000e+01,2.015000e+03,1.200000e+01,3.100000e+01,2.300000e+01,5.900000e+01,0.0


In [18]:
features = features.drop(['second', 'Address'], axis=1)
features.head()

DayOfWeek PdDistrict           X          Y  Year  Month  Day  \
train_df 0  Wednesday   NORTHERN -122.425892  37.774599  2015      5   13   
         1  Wednesday   NORTHERN -122.425892  37.774599  2015      5   13   
         2  Wednesday   NORTHERN -122.424363  37.800414  2015      5   13   
         3  Wednesday   NORTHERN -122.426995  37.800873  2015      5   13   
         4  Wednesday       PARK -122.438738  37.771541  2015      5   13   

            hour  minute  
train_df 0    23      53  
         1    23      53  
         2    23      33  
         3    23      30  
         4    23      30

In [19]:
features['Y'] = features['Y'].replace([90], features['Y'].mode()[0])

features['Y'].describe(include=['O'])

count    1.762311e+06
mean     3.776701e+01
std      2.417117e-02
min      3.770788e+01
25%      3.775239e+01
50%      3.777542e+01
75%      3.778435e+01
max      3.782062e+01
Name: Y, dtype: float64

In [20]:
xyScaler = StandardScaler()
xyScaler.fit(features[['X', 'Y']])
features[['X', 'Y']] = xyScaler.transform(features[['X', 'Y']])

features[['X', 'Y']] = xyScaler.fit_transform(features[['X', 'Y']])

features[['X', 'Y']]

X         Y
train_df 0      -0.105527  0.313905
         1      -0.105527  0.313905
         2      -0.055688  1.381943
         3      -0.141509  1.400904
         4      -0.524343  0.187415
         5       0.632582 -2.216709
         6      -0.021910 -1.732358
         7       1.675162 -1.631989
         8      -2.788829  0.396759
         9       0.116304  1.687565
         10      0.116304  1.687565
         11     -2.129891 -1.214029
         12      0.333877  0.661642
         13     -0.334495  0.717475
         14      0.812158 -1.533912
         15      1.270325 -0.985559
         16      0.327907  0.700044
         17      0.031737 -1.000735
         18      1.181991 -1.159549
         19      0.339240  0.643129
         20     -0.871613 -1.007082
         21      0.077687 -0.814173
         22      0.339240  0.643129
         23      0.339240  0.643129
         24     -0.436692  1.358233
         25      0.395986  0.787388
         26     -0.272646  0.843026
         27     -0.032648 -1.425444
         28     -1.731823 -0.913985
         29     -2.514738 -0.765107
...                   ...       ...
test_df  884232  0.430967 -0.489575
         884233  0.977572  0.636601
         884234 -2.285687  0.251780
         884235 -0.929479 -0.887303
         884236 -0.045055 -0.176233
         884237  0.251743  1.003014
         884238 -1.803653 -1.658030
         884239  0.445732 -0.621571
         884240 -0.690541  1.039479
         884241  0.437979 -1.473587
         884242  0.259899 -0.369601
         884243 -0.433455 -0.356112
         884244  0.486879 -1.756799
         884245  0.290209  0.691688
         884246  1.290094 -1.458713
         884247  0.441117 -0.386135
         884248  0.573469  1.282616
         884249  0.443201  1.227393
         884250  0.627613  0.347917
         884251  0.484066 -0.161834
         884252 -0.743710 -1.470795
         884253  0.389062  0.830970
         884254  0.143965  0.989695
         884255  0.275989 -0.814698
         884256 -0.104626 -0.031340
         884257  0.445732 -0.621571
         884258 -0.087616  1.062018
         884259 -0.742150 -2.272805
         884260  1.149608 -1.139038
         884261 -2.186323 -1.367802

[1762311 rows x 2 columns]

In [21]:
dateScaler = StandardScaler()
dateScaler.fit(features[['Year', 'Month', 'Day', 'hour', 'minute']])
features[['Year', 'Month', 'Day', 'hour', 'minute']] = dateScaler.transform(features[['Year', 'Month', 'Day', 'hour', 'minute']])

features.head()

DayOfWeek PdDistrict         X         Y      Year     Month  \
train_df 0  Wednesday   NORTHERN -0.105527  0.313905  1.731204 -0.398724   
         1  Wednesday   NORTHERN -0.105527  0.313905  1.731204 -0.398724   
         2  Wednesday   NORTHERN -0.055688  1.381943  1.731204 -0.398724   
         3  Wednesday   NORTHERN -0.141509  1.400904  1.731204 -0.398724   
         4  Wednesday       PARK -0.524343  0.187415  1.731204 -0.398724   

                 Day     hour    minute  
train_df 0 -0.295676  1.46404  1.765654  
         1 -0.295676  1.46404  1.765654  
         2 -0.295676  1.46404  0.690542  
         3 -0.295676  1.46404  0.529275  
         4 -0.295676  1.46404  0.529275

In [22]:
leObj = LabelEncoder()
features['DayOfWeek'] = leObj.fit_transform(features['DayOfWeek'])
features['PdDistrict'] = leObj.fit_transform(features['PdDistrict'])

features.head()

DayOfWeek  PdDistrict         X         Y      Year     Month  \
train_df 0          6           4 -0.105527  0.313905  1.731204 -0.398724   
         1          6           4 -0.105527  0.313905  1.731204 -0.398724   
         2          6           4 -0.055688  1.381943  1.731204 -0.398724   
         3          6           4 -0.141509  1.400904  1.731204 -0.398724   
         4          6           5 -0.524343  0.187415  1.731204 -0.398724   

                 Day     hour    minute  
train_df 0 -0.295676  1.46404  1.765654  
         1 -0.295676  1.46404  1.765654  
         2 -0.295676  1.46404  0.690542  
         3 -0.295676  1.46404  0.529275  
         4 -0.295676  1.46404  0.529275

In [23]:
DPScaler = StandardScaler()
features[['DayOfWeek', 'PdDistrict']] = DPScaler.fit_transform(features[['DayOfWeek', 'PdDistrict']])

features.head()

DayOfWeek  PdDistrict         X         Y      Year     Month  \
train_df 0    1.48356   -0.154917 -0.105527  0.313905  1.731204 -0.398724   
         1    1.48356   -0.154917 -0.105527  0.313905  1.731204 -0.398724   
         2    1.48356   -0.154917 -0.055688  1.381943  1.731204 -0.398724   
         3    1.48356   -0.154917 -0.141509  1.400904  1.731204 -0.398724   
         4    1.48356    0.194495 -0.524343  0.187415  1.731204 -0.398724   

                 Day     hour    minute  
train_df 0 -0.295676  1.46404  1.765654  
         1 -0.295676  1.46404  1.765654  
         2 -0.295676  1.46404  0.690542  
         3 -0.295676  1.46404  0.529275  
         4 -0.295676  1.46404  0.529275

In [24]:
features.head()

DayOfWeek  PdDistrict         X         Y      Year     Month  \
train_df 0    1.48356   -0.154917 -0.105527  0.313905  1.731204 -0.398724   
         1    1.48356   -0.154917 -0.105527  0.313905  1.731204 -0.398724   
         2    1.48356   -0.154917 -0.055688  1.381943  1.731204 -0.398724   
         3    1.48356   -0.154917 -0.141509  1.400904  1.731204 -0.398724   
         4    1.48356    0.194495 -0.524343  0.187415  1.731204 -0.398724   

                 Day     hour    minute  
train_df 0 -0.295676  1.46404  1.765654  
         1 -0.295676  1.46404  1.765654  
         2 -0.295676  1.46404  0.690542  
         3 -0.295676  1.46404  0.529275  
         4 -0.295676  1.46404  0.529275

In [25]:
train_columns = train_labels['Category'].unique()
train_columns.sort()
le = LabelEncoder()
le.fit(train_columns)
y_data = le.transform(train_labels['Category'].copy())
y_data

array([37, 21, 21, ..., 16, 35, 12])

In [26]:
x_data = features.loc['train_df']

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((658536, 9), (219513, 9), (658536,), (219513,))

In [3]:
sgd = SGDClassifier(verbose=True)
sgd.fit(x_train, y_train)

print("train set accuracy: {:.3f}".format(sgd.score(x_train, y_train)))
print("test set accuracy: {:.3f}".format(sgd.score(x_test, y_test)))

NameError: name 'x_train' is not defined

In [4]:
sgd.predict(x_train)

NameError: name 'x_train' is not defined

In [ ]:
linear_svm = LinearSVC(verbose=True)
linear_svm.fit(x_train, y_train)
    
print("train set accuracy: {:.3f}".format(linear_svm.score(x_train, y_train)))
print("test set accuracy: {:.3f}".format(linear_svm.score(x_test, y_test)))

[LibLinear]

In [28]:
forest = RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=0, verbose=True)
forest.fit(x_train, y_train)

print("train set accuracy: {:.3f}".format(forest.score(x_train, y_train)))
print("test set accuracy: {:.3f}".format(forest.score(x_test, y_test)))

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.9min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   27.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  1.1min finished


train set accuracy: 0.883


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    9.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   33.7s finished


test set accuracy: 0.316


In [30]:
forest_predict = forest.predict_proba(x_test)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   14.5s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   38.2s finished


In [31]:
gbrt = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10, verbose=True)
gbrt.fit(x_train, y_train)

print("train set accuracy: {:.3f}".format(gbrt.score(x_train, y_train)))
print("test set accuracy: {:.3f}".format(gbrt.score(x_test, y_test)))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1     1705325.1269       46162.4007          318.93m
         2     1610411.2545       23258.2415          305.45m
         3     1542640.6378       16464.5905          298.47m
         4     1491431.9284       12561.1507          293.18m
         5     1450079.9455        9937.6470          286.85m
         6     1417049.1498        7858.8542          282.47m
         7     1390248.5418        6523.4973          278.16m
         8     1366541.9004        5520.2714          273.57m
         9     1345924.7838        4688.1348          268.89m
        10     1328399.2314        4077.9178          264.15m
        20     1235268.5553        1058.5101          227.41m
        30     1198368.3358         338.4788          190.36m


KeyboardInterrupt: 

In [33]:
gbrt_predict = gbrt.predict_proba(x_test)

AttributeError: 'NoneType' object has no attribute 'tree_'

In [37]:
gbrt = GradientBoostingClassifier(n_estimators=10, learning_rate=0.2, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10, verbose=True)
gbrt.fit(x_train, y_train)

print("train set accuracy: {:.3f}".format(gbrt.score(x_train, y_train)))
print("test set accuracy: {:.3f}".format(gbrt.score(x_test, y_test)))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1     1551779.6312       83700.4863           26.82m
         2     1454433.4539       23579.6255           23.70m
         3     1392787.4890       14589.0998           20.54m
         4     1350943.9800        9951.7264           17.38m
         5     1318800.3492        7293.1688           14.24m
         6     1294442.9547        5347.6164           11.30m
         7     1275986.9343        4121.7573            8.37m
         8     1260476.5735        3155.0710            5.66m
         9     1247115.0737        2459.9287            2.82m
        10     1237254.1069        1879.2582            0.00s
train set accuracy: 0.298
test set accuracy: 0.289


In [28]:
rfclf = RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=0, verbose=True)
rfclf.fit(x_data, y_data)
gbrt = GradientBoostingClassifier(n_estimators=10, learning_rate=0.2, verbose=True)
gbrt.fit(x_data, y_data)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.9min finished


      Iter       Train Loss   Remaining Time 
         1     2661196.1900           11.55m
         2     2505212.0982           10.19m
         3     2411464.6685            9.05m
         4     2344890.8178            7.78m
         5     2295383.1817            6.48m
         6     2257654.2734            5.17m
         7     2230483.4678            3.87m
         8     2208448.6552            2.57m
         9     2196965.5766            1.28m
        10     3022315.1830            0.00s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=10, presort='auto', random_state=None,
              subsample=1.0, verbose=True, warm_start=False)

In [29]:
rfclf_predict = rfclf.predict_proba(x_test)
gbclf_predict = gbrt.predict_proba(x_test)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   22.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   51.6s finished


In [33]:
rfclf_predict

array([[ 0.        ,  0.408     ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.74142857,  0.        , ...,  0.01      ,
         0.02666667,  0.        ],
       [ 0.        ,  0.01      ,  0.        , ...,  0.01      ,
         0.01      ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.09      ,
         0.01      ,  0.        ],
       [ 0.        ,  0.02      ,  0.        , ...,  0.        ,
         0.0575    ,  0.        ],
       [ 0.        ,  0.04      ,  0.        , ...,  0.12      ,
         0.        ,  0.01      ]])

In [43]:
train_columns = train_labels['Category'].unique()
train_columns.sort()
le = LabelEncoder()
le.fit(train_columns)
y_data_le = le.transform(train_labels['Category'].copy())

y_data_tf = pd.get_dummies(y_data_le)

In [70]:
# parameters
learning_rate = 0.01  # we can use large learning rate using Batch Normalization
training_epochs = 15
batch_size = 10000
keep_prob = 0.7

hidden_output_size = 256
final_output_size = 39

batch_data = pd.concat([x_data, y_data_tf], axis=1, keys=['x_data', 'y_data'])

batches = np.array_split(batch_data, (len(batch_data) / batch_size))

In [71]:
tf.reset_default_graph()

In [72]:
# input place holders
X = tf.placeholder(tf.float32, [None, 9])
Y = tf.placeholder(tf.float32, [None, 39])
train_mode = tf.placeholder(tf.bool, name='train_mode')

variance_init = tf.contrib.layers.variance_scaling_initializer()
bn_params = {
    'is_training': train_mode,
    'decay': 0.9,
    'updates_collections': None
}

In [73]:
with arg_scope([fully_connected],
               activation_fn=tf.nn.relu,
               weights_initializer=variance_init,
               biases_initializer=None,
               normalizer_fn=batch_norm,
               normalizer_params=bn_params
               ):
    hidden_layer1 = fully_connected(X, hidden_output_size, scope="h1")
    h1_drop = dropout(hidden_layer1, keep_prob, is_training=train_mode)
    hidden_layer2 = fully_connected(h1_drop, hidden_output_size, scope="h2")
    h2_drop = dropout(hidden_layer2, keep_prob, is_training=train_mode)
    hidden_layer3 = fully_connected(h2_drop, hidden_output_size, scope="h3")
    h3_drop = dropout(hidden_layer3, keep_prob, is_training=train_mode)
    hidden_layer4 = fully_connected(h3_drop, hidden_output_size, scope="h4")
    h4_drop = dropout(hidden_layer4, keep_prob, is_training=train_mode)
    hypothesis = fully_connected(h4_drop, final_output_size, activation_fn=None, scope="hypothesis")
    
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [74]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
for epoch in tqdm(range(training_epochs)):
    avg_cost = 0
    total_batch = len(batches)

    for i in tqdm(range(total_batch), leave=False):
        batch_xs = batches[i]['x_data']
        batch_ys = batches[i]['y_data']
        feed_dict_train = {X: batch_xs, Y: batch_ys, train_mode: True}
        feed_dict_cost = {X: batch_xs, Y: batch_ys, train_mode: False}
        opt = sess.run(optimizer, feed_dict=feed_dict_train)
        c = sess.run(cost, feed_dict=feed_dict_cost)
        avg_cost += c / total_batch

    print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost))
    #print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

[Epoch:    1] cost = 3.69793232


[Epoch:    2] cost = 2.73660024


[Epoch:    3] cost = 2.6075505


[Epoch:    4] cost = 2.55274229


[Epoch:    5] cost = 2.52147858


...

[Epoch:   11] cost = 2.46963471


[Epoch:   12] cost = 2.46509916


[Epoch:   13] cost = 2.46262455


[Epoch:   14] cost = 2.46041301


[Epoch:   15] cost = 2.4579947

Learning Finished!


In [75]:
test_data = features.loc['test_df']
test_data

,DayOfWeek,PdDistrict,X,Y,Year,Month,Day,hour,minute
0,0.003516,-1.552569,0.752061,-1.322242,1.731204,-0.398724,-0.635456,1.464040,2.088188
1,0.003516,-1.552569,1.014998,-1.430579,1.731204,-0.398724,-0.635456,1.464040,1.658143
2,0.003516,-0.154917,-0.109122,1.042618,1.731204,-0.398724,-0.635456,1.464040,1.604387
3,0.003516,-0.853743,-0.480536,-1.886507,1.731204,-0.398724,-0.635456,1.464040,1.335609
4,0.003516,-0.853743,-0.480536,-1.886507,1.731204,-0.398724,-0.635456,1.464040,1.335609
5,0.003516,1.242734,-1.185727,-2.227416,1.731204,-0.398724,-0.635456,1.464040,1.066831
6,0.003516,-0.853743,-0.096553,-1.144365,1.731204,-0.398724,-0.635456,1.464040,0.529275
7,0.003516,-0.853743,0.326125,-1.127831,1.731204,-0.398724,-0.635456,1.464040,0.529275
8,0.003516,-0.504330,0.128940,-0.076380,1.731204,-0.398724,-0.635456,1.464040,-0.545837
9,0.003516,-1.203156,0.284310,1.318733,1.731204,-0.398724,-0.635456,1.464040,-0.545837


In [76]:
resHypo = tf.nn.softmax(hypothesis)
res = sess.run(resHypo, feed_dict={X: test_data, train_mode: False})

array([[ 0.0067243 ,  0.14944574,  0.00031053, ...,  0.03574138,
         0.08579698,  0.03106871],
       [ 0.0076915 ,  0.1515207 ,  0.00031518, ...,  0.03484092,
         0.08937467,  0.03329136],
       [ 0.00198316,  0.10466219,  0.00042759, ...,  0.04061434,
         0.07694858,  0.01062273],
       ..., 
       [ 0.00203695,  0.09871749,  0.00191186, ...,  0.16081251,
         0.01411227,  0.00656969],
       [ 0.0032817 ,  0.11604244,  0.0018856 , ...,  0.128433  ,
         0.02087542,  0.00920838],
       [ 0.00137005,  0.07071145,  0.00222588, ...,  0.14423071,
         0.0119176 ,  0.00419746]], dtype=float32)

In [81]:
rfclf_predict = rfclf.predict_proba(test_data)
gbclf_predict = gbrt.predict_proba(test_data)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   39.5s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  1.6min finished


In [82]:
res.shape, rfclf_predict.shape, gbclf_predict.shape

((884262, 39), (884262, 39), (884262, 39))

In [83]:
resData = (res + rfclf_predict + gbclf_predict) / 3
resData

array([[ 0.00734911,  0.13361569,  0.00114983, ...,  0.03194169,
         0.06452923,  0.02877363],
       [ 0.00453575,  0.10856967,  0.00112806, ...,  0.02151982,
         0.09992543,  0.02443546],
       [ 0.008526  ,  0.08860623,  0.00104153, ...,  0.01984903,
         0.05295017,  0.01736937],
       ..., 
       [ 0.00221774,  0.07798891,  0.00208456, ...,  0.13271043,
         0.01920377,  0.00495013],
       [ 0.00657903,  0.07795643,  0.00219405, ...,  0.106765  ,
         0.02275355,  0.00599185],
       [ 0.0019847 ,  0.07292045,  0.00213411, ...,  0.09977114,
         0.01163415,  0.00375333]])

In [87]:
train_columns

array(['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
       'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
       'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
       'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
       'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
       'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
       'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
       'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
       'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
       'WARRANTS', 'WEAPON LAWS'], dtype=object)

In [89]:
res_df = pd.DataFrame(data=resData, index=test_id, columns=train_columns)

In [90]:
res_df.to_csv('./result.csv')